In [2]:
import numpy as np
import revreaddy as rdy
import logging
import datetime
import h5py as h5
import matplotlib.cm as cm

import time
import random
import string

import matplotlib.pyplot as plt
reload(logging)
logging.basicConfig(
     format='%(asctime)s %(levelname)s: %(message)s',
     datefmt='%Y/%m/%d %I:%M:%S',
     level=logging.DEBUG
    )

In [3]:
def stamp():
    
    
    timestamp = time.strftime("%Y_%m_%d-%H_%M_%S")
    randomstamp = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(6))
    stamp = timestamp + "_" + randomstamp
    return stamp



make envirement 


In [4]:
def envirement(reactiondistance):
    positions=[]
    posenzyme = [0,0,0]
    amountenzyme = 0
    while amountenzyme < 1000:
        #todo periodic boundaryconditions !!!!
        pos = np.random.random(3) * 19.999 - 9.999
        distance=np.sqrt((posenzyme[0]-pos[0])**2+(posenzyme[1]-pos[1])**2+(posenzyme[2]-pos[2])**2)
        if distance > reactiondistance:
            amountenzyme+=1
            positions.append(pos)
    positions=np.array(positions)
    return positions,posenzyme
         
        
        

In [5]:
#visuallisierung von dem zufällig vereilten Punkten im Raum
from mpl_toolkits.mplot3d import Axes3D
positions,enzymepos= envirement(0.7)
fig = plt.figure()
ax = plt.axes(projection='3d')

z = positions[:,0]
x = positions[:,1]
y = positions[:,2]

c = x + y

ax.scatter(x, y, z,c="b")
ax.scatter(enzymepos[1],enzymepos[2],enzymepos[0],c="r")

plt.show(fig)

In [6]:
def simulationfrac(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    print "fractional" 
    sim = rdy.Sim("FractionalDiffusion")
    print "FractionalDiffusion"
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1 # ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0.1, reactiondistance)
    sim.delete_all_particles()
    for i in range(500):
        pos = positions[i,:]
        sim.add_particle(pos, 0)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 1)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 2)
    for _ in range(1):      
        sim.add_particle(posenzyme, 2)
    sim.show_world()
    sim.delete_all_observables()
    numbers_a_name = stamp()+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    numbers_b_name = stamp()+"numbers_b_test1.h5"
    mean_e_name = stamp()+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    print "NUMBERS A:", numbers_a_name
    print "NUMBERS B:", numbers_b_name
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(3000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb, fc


In [7]:
def simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1 # ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0.1, reactiondistance)
    sim.delete_all_particles()
    for i in range(500):
        pos = positions[i,:]
        sim.add_particle(pos, 0)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 1)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 2)
    for _ in range(1):      
        sim.add_particle(posenzyme, 2)
    sim.show_world()
    sim.delete_all_observables()
    numbers_a_name = stamp()+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    numbers_b_name = stamp()+"numbers_b_test1.h5"
    mean_e_name = stamp()+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    print "NUMBERS A:", numbers_a_name
    print "NUMBERS B:", numbers_b_name
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(3000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb, fc


In [8]:
#hier kommt die berechnung des verhältnisses der reactionskoeffizienten und der Concentration vom enzym
def analyt(t,boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction):
    #a=((4*np.pi*reactiondistance**3)*(Diffusion_E+Diffusion_S))/((3*boxsize**3)*(Diffusion_E+Diffusion_P))
    k1=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreaction)+np.tanh(reactiondistance*np.sqrt(intrinsikreaction/(Diffusion_E+Diffusion_S))))
    #t=np.linspace(1,40)
    #vor und rück reaction möglich                                        
    #c=np.exp(-t*a)*fa['particleNumbers'][0]
    #nur vor reaction
    k11=intrinsikreaction*4*np.pi*reactiondistance**3/3
    c=fa['particleNumbers'][2]*(np.exp((-t*k1*1)/(boxsize**3)))
    c11=fa['particleNumbers'][0]*np.exp((-t*k11*1)/(boxsize**3))

    return t, c,c11

target search problem: intrinsic reaction rate = infinity , fractional vs browian motion Particle number comparision.  Starting scenario: 1 Enzyme, 200 Substrate, boxsize 20, Diffusion coefficient of substrate and enzyme = 1.5

In [14]:


colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [0.2]:
    boxsize=30.0
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=0.2
    reactiondistance=2.0
    intrinsikreaction=0.5
    colornow=next(colors)
    falist=[]
    fclist=[]
    faflist=[]
    averagelist=[]
    positions,posenzyme =  envirement(reactiondistance)
    for ii in range(15):
        print ii
        fa,fb,fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc
        faf,fbf,fcf=simulationfrac(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc
        falist.append(fa['particleNumbers'])
        faflist.append(faf['particleNumbers'])   
    falist=np.array(falist) 
    faflist=np.array(faflist)
    paertnum=falist.mean(axis=0) 
    paertnumf=faflist.mean(axis=0) 
    plt.plot(fbf['times'], paertnumf,color=colornow,label="$D_S+D_E$=$%.2f$ fractional" %(2*i))
    plt.plot(fb['times'], paertnum,color=next(colors),label="$D_S+D_E$=$%.2f$ brownian" %(2*i))
plt.ylabel('S(t)', fontsize=10)
plt.xlabel('t in ns', fontsize=10)
plt.legend()
plt.show()

2016/08/29 01:40:48 INFO: There are either no particles or too much (>200) to list them here.
2016/08/29 01:40:48 INFO: Run with timestep 1.0 and 3000 timesteps
2016/08/29 01:40:51 INFO: Finished after 2.726093 seconds.


0
Number of particles:  501
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 1                  
 
NUMBERS A: 2016_08_29-13_40_48_698EGZnumbers_a_test1.h5
NUMBERS B: 2016_08_29-13_40_48_AWQ5PDnumbers_b_test1.h5
fractional

2016/08/29 01:40:51 INFO: There are either no particles or too much (>200) to list them here.
2016/08/29 01:40:51 INFO: Run with timestep 1.0 and 3000 timesteps



FractionalDiffusion
Number of particles:  501
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 1                  
 
NUMBERS A: 2016_08_29-13_40_51_FPSNXWnumbers_a_test1.h5
NUMBERS B: 2016_08_29-13_40_51_LCA5CUnumbers_b_test1.h5


KeyboardInterrupt: 

meansquaredisplacment enzyme 


In [9]:
def msd_analyt(alpha,dt,t,diffconst):
    t=np.linspace(0,t,t)
    return 2*3*((t*dt)**alpha)*diffconst

In [8]:
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
msd_text='MSD=6 K_\\alpha t^\\alpha'
msd_text1='MSD=6 D t'

for i in [0.2]:
    boxsize=30.0
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=0.2
    reactiondistance=2.0
    intrinsikreaction=0.5
    colornow=next(colors)
    falist=[]
    fclist=[]
    fcflist=[]
    averagelist=[]
    positions,posenzyme =  envirement(reactiondistance)
    for ii in range(10):
        print ii
        fa,fb,fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc
        faf,fbf,fcf=simulationfrac(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc
        fclist.append(fc['meanSquaredDisplacements'])
        fcflist.append(fcf['meanSquaredDisplacements'])   
    fclist=np.array(fclist) 
    fcflist=np.array(fcflist)
    msd=fclist.mean(axis=0) 
    msdf=fcflist.mean(axis=0) 
    plt.plot(fcf['times'], msdf,color=colornow,label="$D_S+D_E$=$%.2f$ fractional" %(2*i))
    plt.plot(fc['times'],msd,color=next(colors),label="$D_S+D_E$=$%.2f$ brownian" %(2*i))
plt.loglog(np.array(range(3000)),msd_analyt(0.5,1,3000,0.2),label="$%s$  " %(msd_text),linestyle='--')
plt.loglog(np.array(range(3000)),msd_analyt(1,1,3000,0.2),label="$%s$  " %(msd_text1),linestyle='--')
plt.ylabel('S(t)', fontsize=10)
plt.xlabel('t in ns', fontsize=10)
plt.legend()
plt.show()

2016/08/09 07:06:34 INFO: There are either no particles or too much (>200) to list them here.
2016/08/09 07:06:34 INFO: Run with timestep 1.0 and 3000 timesteps
2016/08/09 07:06:37 INFO: Finished after 2.809486 seconds.


0
Number of particles:  501
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 1                  
 
NUMBERS A: 2016_08_09-19_06_34_MWSHBLnumbers_a_test1.h5
NUMBERS B: 2016_08_09-19_06_34_3WKK8Wnumbers_b_test1.h5
fractional

2016/08/09 07:06:37 INFO: There are either no particles or too much (>200) to list them here.
2016/08/09 07:06:37 INFO: Run with timestep 1.0 and 3000 timesteps
2016/08/09 07:06:56 INFO: Finished after 19.422037 seconds.



FractionalDiffusion
Number of particles:  501
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 1                  
 
NUMBERS A: 2016_08_09-19_06_37_REFLZ5numbers_a_test1.h5
NUMBERS B: 2016_08_09-19_06_37_U6QV97numbers_b_test1.h5
1

2016/08/09 07:06:56 INFO: There are either no particles or too much (>200) to list them here.
2016/08/09 07:06:56 INFO: Run with timestep 1.0 and 3000 timesteps
2016/08/09 07:06:59 INFO: Finished after 2.942474 seconds.


...


FractionalDiffusion
Number of particles:  501
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 1                  
 
NUMBERS A: 2016_08_09-19_09_34_UGGHJ1numbers_a_test1.h5
NUMBERS B: 2016_08_09-19_09_34_9A5V5Vnumbers_b_test1.h5
9

2016/08/09 07:09:54 INFO: There are either no particles or too much (>200) to list them here.
2016/08/09 07:09:54 INFO: Run with timestep 1.0 and 3000 timesteps
2016/08/09 07:09:56 INFO: Finished after 2.758181 seconds.



Number of particles:  501
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 1                  
 
NUMBERS A: 2016_08_09-19_09_54_REN87Gnumbers_a_test1.h5
NUMBERS B: 2016_08_09-19_09_54_RB1U3Qnumbers_b_test1.h5
fractional

2016/08/09 07:09:56 INFO: There are either no particles or too much (>200) to list them here.
2016/08/09 07:09:56 INFO: Run with timestep 1.0 and 3000 timesteps
2016/08/09 07:10:15 INFO: Finished after 18.924241 seconds.



FractionalDiffusion
Number of particles:  501
TypeId | Number of particles
====== | ===================
0      | 500                
1      | 0                  
2      | 1                  
 
NUMBERS A: 2016_08_09-19_09_56_X64E7Snumbers_a_test1.h5
NUMBERS B: 2016_08_09-19_09_56_JLS1OJnumbers_b_test1.h5


radial dirtribution function

In [10]:
def simulation2(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim("FractionalDiffusion")
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1.0# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0.1, reactiondistance)
    sim.delete_all_particles()
    for i in range(500):
        pos = positions[i,:]
        sim.add_particle(pos, 0)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 1)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 2)
    for _ in range(1):      
        sim.add_particle(posenzyme, 2)
    sim.show_world()
    sim.delete_all_observables()
    mean_e_name = stamp()+"radialdist_e_test1.h5"
    sim.new_radial_distribution(2000,mean_e_name,np.arange(0., 5., 0.05),np.array([[0,1]]))
    sim.run(2000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fc = h5.File(mean_e_name, 'r')
    return  fc
def simulation3(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1.0# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0.1, reactiondistance)
    sim.delete_all_particles()
    for i in range(500):
        pos = positions[i,:]
        sim.add_particle(pos, 0)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 1)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 2)
    for _ in range(1):      
        sim.add_particle(posenzyme, 2)
    sim.show_world()
    sim.delete_all_observables()
    mean_e_name = stamp()+"radialdist_e_test1.h5"
    sim.new_radial_distribution(2000,mean_e_name,np.arange(0., 5., 0.05),np.array([[0,1]]))
    sim.run(2000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fc = h5.File(mean_e_name, 'r')
    return  fc

In [11]:
positions,posenzyme =  envirement(reactiondistance)
fd=simulation2(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) 
fc=simulation3(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) 


#,fc
plt.plot(fd['binCenters'],fd['bins'], label="fractional")
plt.plot(fc['binCenters'],fc['bins'], label="brownian")

plt.ylabel('Bins', fontsize=10)
plt.xlabel('Bin-Centers', fontsize=10)
plt.legend()
#plt.savefig('finalreport/data/fractionalradialdistributionrevreaddy.png',dpi=300)   # save the figure to file 
plt.show()

NameError: name 'reactiondistance' is not defined

trajectory

In [85]:
def simulationtrajectory(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1.0# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0.1, reactiondistance)
    sim.delete_all_particles()
    for i in range(500):
        pos = positions[i,:]
        sim.add_particle(pos, 0)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 1)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 2)
    for _ in range(1):      
        sim.add_particle(posenzyme, 2)
    sim.show_world()
    sim.delete_all_observables()
    mean_e_name = stamp()+"radialdist_e_test1.h5"
    sim.new_trajectory(1,mean_e_name)
    sim.run(2000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fc = h5.File(mean_e_name, 'r')
    return  fc
def simulationtrajectory2(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim("FractionalDiffusion")
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 1.0# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0.1, reactiondistance)
    sim.delete_all_particles()
    for i in range(500):
        pos = positions[i,:]
        sim.add_particle(pos, 0)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 1)
        #pos = np.random.random(3) * 9.9 - 4.95
        #sim.add_particle(pos, 2)
    for _ in range(1):      
        sim.add_particle(posenzyme, 2)
    sim.delete_all_observables()
    sim.run(4096)
    num_particles = sim.world.getNumberOfParticles()  
    result = []
    result2 = []
    for i in range(num_particles):
            if sim.world.getTypeId(i) == 0:
                pos = sim.world.getPosition(i)
                result.append(pos)
            elif sim.world.getTypeId(i) == 1:
                pos = sim.world.getPosition(i)
                result2.append(pos)
    return  np.array(result),np.array(result2)

In [86]:
#trajectory=simulationtrajectory(100,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) 
num_particles,num_particles2=simulationtrajectory2(100,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) 

2016/08/29 02:52:37 INFO: Run with timestep 1.0 and 4096 timesteps
2016/08/29 02:54:03 INFO: Finished after 86.497229 seconds.


In [87]:
print num_particles.shape
print num_particles2.shape


(338, 3)
(162, 3)


In [88]:
print 
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(num_particles[:,0], num_particles[:,1],num_particles[:,2], c='b')
ax.scatter(num_particles2[:,0], num_particles2[:,1], num_particles2[:,2], c='r')
plt.show()

In [52]:
st=str(traj['typeIds'][0])+"    "+str(traj['positions'][length,particle,0])+"    "+str(traj['positions'][length,particle,1])+"    "+str(traj['positions'][length,particle,2])+'\n')

st=str(traj['typeIds'][0])+"    "+str(traj['positions'][length,particle,0])+"    "+str(traj['positions'][length,particle,1])+"    "+str(traj['positions'][length,particle,2])+'\n')
print s

<501> /n


In [16]:
sim = rdy.Sim("FractionalDiffusion")
sim.new_trajectory()

In [48]:
trajectory['typeIds'][0]


0

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(trajectory['positions'][:,500,0], trajectory['positions'][:,500,1], trajectory['positions'][:,500,2], '-b')
ax.plot(trajectory2['positions'][:,500,0], trajectory2['positions'][:,500,1], trajectory2['positions'][:,500,2], '-r')
plt.show()

260.0